# PREDICTIVE MAINTAINANCE

In [108]:
import pandas as pd 
import numpy as np
import os 
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
import prophet


from sklearn.preprocessing import LabelEncoder,StandardScaler
# import ProfileReport from pandas_profiling

#TS 
from statsmodels.tsa.vector_ar.var_model import VAR

#Classification
from xgboost import XGBClassifier
from joblib import dump, load
# from sklearn.externals import joblib
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report,roc_curve, auc,confusion_matrix
from imblearn.over_sampling import SMOTE
from collections import Counter


In [128]:
data = pd.read_csv("..\Predictive_Maintainance_V1\sensor.csv")
data = data.drop(['Unnamed: 0'], axis=1)
data

<>:1: SyntaxWarning: invalid escape sequence '\P'
<>:1: SyntaxWarning: invalid escape sequence '\P'
C:\Users\Jayasree\AppData\Local\Temp\ipykernel_14264\2508845951.py:1: SyntaxWarning: invalid escape sequence '\P'
  data = pd.read_csv("..\Predictive_Maintainance_V1\sensor.csv")


,timestamp,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,...,sensor_43,sensor_44,sensor_45,sensor_46,sensor_47,sensor_48,sensor_49,sensor_50,sensor_51,machine_status
0,2018-04-01 00:00:00,2.465394,47.09201,53.211800,46.310760,634.375000,76.45975,13.41146,16.13136,15.56713,...,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,243.0556,201.3889,NORMAL
1,2018-04-01 00:01:00,2.465394,47.09201,53.211800,46.310760,634.375000,76.45975,13.41146,16.13136,15.56713,...,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,243.0556,201.3889,NORMAL
2,2018-04-01 00:02:00,2.444734,47.35243,53.211800,46.397570,638.888900,73.54598,13.32465,16.03733,15.61777,...,41.66666,39.351852,65.39352,51.21528,38.194443,155.9606,67.12963,241.3194,203.7037,NORMAL
3,2018-04-01 00:03:00,2.460474,47.09201,53.168400,46.397568,628.125000,76.98898,13.31742,16.24711,15.69734,...,40.88541,39.062500,64.81481,51.21528,38.194440,155.9606,66.84028,240.4514,203.1250,NORMAL
4,2018-04-01 00:04:00,2.445718,47.13541,53.211800,46.397568,636.458300,76.58897,13.35359,16.21094,15.69734,...,41.40625,38.773150,65.10416,51.79398,38.773150,158.2755,66.55093,242.1875,201.3889,NORMAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220315,2018-08-31 23:55:00,2.407350,47.69965,50.520830,43.142361,634.722229,64.59095,15.11863,16.65220,15.65393,...,38.28125,68.287030,52.37268,48.32176,41.087960,212.3843,153.64580,NaN,231.1921,NORMAL
220316,2018-08-31 23:56:00,2.400463,47.69965,50.564240,43.142361,630.902771,65.83363,15.15480,16.70284,15.65393,...,38.28125,66.840280,50.63657,48.03241,40.798610,213.8310,156.25000,NaN,231.1921,NORMAL
220317,2018-08-31 23:57:00,2.396528,47.69965,50.520830,43.142361,625.925903,67.29445,15.08970,16.70284,15.69734,...,39.06250,65.393520,48.90046,48.03241,40.798610,217.3032,155.38190,NaN,232.0602,NORMAL
220318,2018-08-31 23:58:00,2.406366,47.69965,50.520832,43.142361,635.648100,65.09175,15.11863,16.56539,15.74074,...,40.62500,64.236110,47.74306,48.32176,40.509258,222.5116,153.93520,NaN,234.0856,NORMAL


In [63]:
data['machine_status'].value_counts()

machine_status
NORMAL        205836
RECOVERING     14477
BROKEN             7
Name: count, dtype: int64

In [64]:
# Replace 'RECOVERING' with 'BROKEN'
data['machine_status'] = data['machine_status'].replace('RECOVERING', 'BROKEN')

# ENcode the machine Status Column
# label_encoder = LabelEncoder()
# data['machine_status'] = label_encoder.fit_transform(data['machine_status'])
data['machine_status'] = data['machine_status'].map({'NORMAL': 0, 'BROKEN': 1})



In [65]:
missing_percentage = (data.isnull().mean() * 100).sort_values(ascending=False)
columns_to_drop = missing_percentage[missing_percentage > 30].index
data = data.drop(columns=columns_to_drop)
data

,timestamp,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,...,sensor_42,sensor_43,sensor_44,sensor_45,sensor_46,sensor_47,sensor_48,sensor_49,sensor_51,machine_status
0,2018-04-01 00:00:00,2.465394,47.09201,53.211800,46.310760,634.375000,76.45975,13.41146,16.13136,15.56713,...,31.770832,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,201.3889,0
1,2018-04-01 00:01:00,2.465394,47.09201,53.211800,46.310760,634.375000,76.45975,13.41146,16.13136,15.56713,...,31.770832,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,201.3889,0
2,2018-04-01 00:02:00,2.444734,47.35243,53.211800,46.397570,638.888900,73.54598,13.32465,16.03733,15.61777,...,31.770830,41.66666,39.351852,65.39352,51.21528,38.194443,155.9606,67.12963,203.7037,0
3,2018-04-01 00:03:00,2.460474,47.09201,53.168400,46.397568,628.125000,76.98898,13.31742,16.24711,15.69734,...,31.510420,40.88541,39.062500,64.81481,51.21528,38.194440,155.9606,66.84028,203.1250,0
4,2018-04-01 00:04:00,2.445718,47.13541,53.211800,46.397568,636.458300,76.58897,13.35359,16.21094,15.69734,...,31.510420,41.40625,38.773150,65.10416,51.79398,38.773150,158.2755,66.55093,201.3889,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220315,2018-08-31 23:55:00,2.407350,47.69965,50.520830,43.142361,634.722229,64.59095,15.11863,16.65220,15.65393,...,30.208330,38.28125,68.287030,52.37268,48.32176,41.087960,212.3843,153.64580,231.1921,0
220316,2018-08-31 23:56:00,2.400463,47.69965,50.564240,43.142361,630.902771,65.83363,15.15480,16.70284,15.65393,...,29.947920,38.28125,66.840280,50.63657,48.03241,40.798610,213.8310,156.25000,231.1921,0
220317,2018-08-31 23:57:00,2.396528,47.69965,50.520830,43.142361,625.925903,67.29445,15.08970,16.70284,15.69734,...,30.208330,39.06250,65.393520,48.90046,48.03241,40.798610,217.3032,155.38190,232.0602,0
220318,2018-08-31 23:58:00,2.406366,47.69965,50.520832,43.142361,635.648100,65.09175,15.11863,16.56539,15.74074,...,30.208332,40.62500,64.236110,47.74306,48.32176,40.509258,222.5116,153.93520,234.0856,0


In [66]:
data['timestamp'] = pd.to_datetime(data['timestamp'])
# mean_values = data.groupby('timestamp').mean()

# data = data.groupby('timestamp').apply(lambda group: group.fillna(group.mean()))
data = data.fillna(data.mean())


In [67]:
data

,timestamp,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,...,sensor_42,sensor_43,sensor_44,sensor_45,sensor_46,sensor_47,sensor_48,sensor_49,sensor_51,machine_status
0,2018-04-01 00:00:00,2.465394,47.09201,53.211800,46.310760,634.375000,76.45975,13.41146,16.13136,15.56713,...,31.770832,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,201.3889,0
1,2018-04-01 00:01:00,2.465394,47.09201,53.211800,46.310760,634.375000,76.45975,13.41146,16.13136,15.56713,...,31.770832,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,201.3889,0
2,2018-04-01 00:02:00,2.444734,47.35243,53.211800,46.397570,638.888900,73.54598,13.32465,16.03733,15.61777,...,31.770830,41.66666,39.351852,65.39352,51.21528,38.194443,155.9606,67.12963,203.7037,0
3,2018-04-01 00:03:00,2.460474,47.09201,53.168400,46.397568,628.125000,76.98898,13.31742,16.24711,15.69734,...,31.510420,40.88541,39.062500,64.81481,51.21528,38.194440,155.9606,66.84028,203.1250,0
4,2018-04-01 00:04:00,2.445718,47.13541,53.211800,46.397568,636.458300,76.58897,13.35359,16.21094,15.69734,...,31.510420,41.40625,38.773150,65.10416,51.79398,38.773150,158.2755,66.55093,201.3889,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220315,2018-08-31 23:55:00,2.407350,47.69965,50.520830,43.142361,634.722229,64.59095,15.11863,16.65220,15.65393,...,30.208330,38.28125,68.287030,52.37268,48.32176,41.087960,212.3843,153.64580,231.1921,0
220316,2018-08-31 23:56:00,2.400463,47.69965,50.564240,43.142361,630.902771,65.83363,15.15480,16.70284,15.65393,...,29.947920,38.28125,66.840280,50.63657,48.03241,40.798610,213.8310,156.25000,231.1921,0
220317,2018-08-31 23:57:00,2.396528,47.69965,50.520830,43.142361,625.925903,67.29445,15.08970,16.70284,15.69734,...,30.208330,39.06250,65.393520,48.90046,48.03241,40.798610,217.3032,155.38190,232.0602,0
220318,2018-08-31 23:58:00,2.406366,47.69965,50.520832,43.142361,635.648100,65.09175,15.11863,16.56539,15.74074,...,30.208332,40.62500,64.236110,47.74306,48.32176,40.509258,222.5116,153.93520,234.0856,0


# FORECASTING MODEL|

In [68]:
train_data = data[:218880]
test_data = data[218880:]

In [69]:
var_df = train_data.set_index('timestamp')

In [70]:
var_df.columns

Index(['sensor_00', 'sensor_01', 'sensor_02', 'sensor_03', 'sensor_04',
       'sensor_05', 'sensor_06', 'sensor_07', 'sensor_08', 'sensor_09',
       'sensor_10', 'sensor_11', 'sensor_12', 'sensor_13', 'sensor_14',
       'sensor_16', 'sensor_17', 'sensor_18', 'sensor_19', 'sensor_20',
       'sensor_21', 'sensor_22', 'sensor_23', 'sensor_24', 'sensor_25',
       'sensor_26', 'sensor_27', 'sensor_28', 'sensor_29', 'sensor_30',
       'sensor_31', 'sensor_32', 'sensor_33', 'sensor_34', 'sensor_35',
       'sensor_36', 'sensor_37', 'sensor_38', 'sensor_39', 'sensor_40',
       'sensor_41', 'sensor_42', 'sensor_43', 'sensor_44', 'sensor_45',
       'sensor_46', 'sensor_47', 'sensor_48', 'sensor_49', 'sensor_51',
       'machine_status'],
      dtype='object')

In [71]:
model = VAR(var_df)
model_fit = model.fit()

d:\Predictive_Maintainance\Predictive_Maintainance_V1\Project_Env\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency T will be used.
  self._init_dates(dates, freq)


In [72]:
model_fit.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Sat, 23, Dec, 2023
Time:                     23:27:50
--------------------------------------------------------------------
No. of Equations:         51.0000    BIC:                    65.2484
Nobs:                     218879.    HQIC:                   65.1602
Log likelihood:      -2.29638e+07    FPE:                1.91794e+28
AIC:                      65.1236    Det(Omega_mle):     1.89485e+28
--------------------------------------------------------------------
Results for equation sensor_00
                       coefficient       std. error           t-stat            prob
------------------------------------------------------------------------------------
const                     0.015482         0.001740            8.898           0.000
L1.sensor_00              0.998101         0.000218         4588.299           0.000
L1.sensor_01             -0.000043      

In [73]:
future = pd.DataFrame(test_data['timestamp']).reset_index()
del future['index']

In [74]:
lag_order = model_fit.k_ar
forecast = model_fit.forecast(model_fit.endog, steps=1440)

In [75]:
forecast.shape

(1440, 51)

In [76]:
forecast_df = pd.DataFrame(forecast, columns=var_df.columns)

In [77]:
forecast_df = pd.merge(forecast_df, future, left_index=True, right_index=True)

In [78]:
columns = ['timestamp', 'sensor_00', 'sensor_01', 'sensor_02', 'sensor_03', 'sensor_04',
                'sensor_05', 'sensor_06', 'sensor_07', 'sensor_08', 'sensor_09',
                'sensor_10', 'sensor_11', 'sensor_12', 'sensor_13', 'sensor_14',
                'sensor_16', 'sensor_17', 'sensor_18', 'sensor_19', 'sensor_20',
                'sensor_21', 'sensor_22', 'sensor_23', 'sensor_24', 'sensor_25',
                'sensor_26', 'sensor_27', 'sensor_28', 'sensor_29', 'sensor_30',
                'sensor_31', 'sensor_32', 'sensor_33', 'sensor_34', 'sensor_35',
                'sensor_36', 'sensor_37', 'sensor_38', 'sensor_39', 'sensor_40',
                'sensor_41', 'sensor_42', 'sensor_43', 'sensor_44', 'sensor_45',
                'sensor_46', 'sensor_47', 'sensor_48', 'sensor_49', 'sensor_51',
                'machine_status']


forecast_df = forecast_df[columns]

forecast_df.columns

Index(['timestamp', 'sensor_00', 'sensor_01', 'sensor_02', 'sensor_03',
       'sensor_04', 'sensor_05', 'sensor_06', 'sensor_07', 'sensor_08',
       'sensor_09', 'sensor_10', 'sensor_11', 'sensor_12', 'sensor_13',
       'sensor_14', 'sensor_16', 'sensor_17', 'sensor_18', 'sensor_19',
       'sensor_20', 'sensor_21', 'sensor_22', 'sensor_23', 'sensor_24',
       'sensor_25', 'sensor_26', 'sensor_27', 'sensor_28', 'sensor_29',
       'sensor_30', 'sensor_31', 'sensor_32', 'sensor_33', 'sensor_34',
       'sensor_35', 'sensor_36', 'sensor_37', 'sensor_38', 'sensor_39',
       'sensor_40', 'sensor_41', 'sensor_42', 'sensor_43', 'sensor_44',
       'sensor_45', 'sensor_46', 'sensor_47', 'sensor_48', 'sensor_49',
       'sensor_51', 'machine_status'],
      dtype='object')

In [79]:
forecast_df

,timestamp,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,...,sensor_42,sensor_43,sensor_44,sensor_45,sensor_46,sensor_47,sensor_48,sensor_49,sensor_51,machine_status
0,2018-08-31 00:00:00,2.405034,48.450161,52.167524,44.965660,616.909028,74.887820,15.114038,16.694029,15.765068,...,32.538144,41.313815,56.063739,55.725431,55.801229,42.113511,306.800147,112.752672,208.579973,-0.000344
1,2018-08-31 00:01:00,2.405647,48.462831,52.164947,44.966022,616.611914,74.898851,15.109299,16.685692,15.760704,...,32.527212,41.232487,55.724637,55.322427,55.488402,41.995065,306.300172,112.362912,208.533062,-0.000676
2,2018-08-31 00:02:00,2.406239,48.475471,52.162402,44.966362,616.351404,74.908705,15.104431,16.677804,15.756568,...,32.519207,41.161941,55.404589,54.925707,55.195593,41.889315,305.791734,111.968040,208.482303,-0.000996
3,2018-08-31 00:03:00,2.406809,48.488049,52.159885,44.966684,616.125033,74.917276,15.099454,16.670343,15.752646,...,32.514050,41.101836,55.102113,54.535734,54.922609,41.795590,305.274870,111.568702,208.427990,-0.001306
4,2018-08-31 00:04:00,2.407358,48.500538,52.157393,44.966988,615.930505,74.924488,15.094381,16.663286,15.748924,...,32.511657,41.051816,54.815920,54.152979,54.669245,41.713256,304.749684,111.165484,208.370382,-0.001605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,2018-08-31 23:55:00,2.406768,48.419529,51.721246,44.471804,627.534583,73.833800,14.007350,16.194893,15.500580,...,35.495831,44.814032,44.611633,44.825750,50.590559,45.891003,179.774081,60.973521,192.523459,0.004023
1436,2018-08-31 23:56:00,2.406777,48.419274,51.721058,44.471487,627.528376,73.834594,14.007196,16.194806,15.500514,...,35.495688,44.813501,44.610494,44.824694,50.589099,45.890489,179.757182,60.970982,192.529634,0.004032
1437,2018-08-31 23:57:00,2.406786,48.419019,51.720871,44.471170,627.522165,73.835387,14.007042,16.194720,15.500447,...,35.495544,44.812971,44.609356,44.823639,50.587641,45.889976,179.740304,60.968447,192.535810,0.004042
1438,2018-08-31 23:58:00,2.406795,48.418764,51.720683,44.470854,627.515950,73.836179,14.006888,16.194633,15.500380,...,35.495401,44.812442,44.608219,44.822585,50.586185,45.889462,179.723447,60.965915,192.541986,0.004051


# CLASSIFICATION MODEL

In [80]:
# data = data.drop('timestamp', axis=1)

X = data.drop(['machine_status','timestamp'], axis=1)
y = data['machine_status']
forecasted_X = forecast_df.drop(['machine_status','timestamp'], axis=1)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)




In [81]:
sm=SMOTE(random_state=0)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)
print("The number of classes before fit {}".format(Counter(y_train)))
print("The number of classes after fit {}".format(Counter(y_train_res)))

The number of classes before fit Counter({0: 164593, 1: 11663})
The number of classes after fit Counter({0: 164593, 1: 164593})


In [82]:
logreg = LogisticRegression()
logreg.fit(X_train_res, y_train_res)

d:\Predictive_Maintainance\Predictive_Maintainance_V1\Project_Env\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [83]:
y_pred2 = logreg.predict(X_test)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred2))
print("Accuracy Score:", accuracy_score(y_test, y_pred2))
print("Classification Report:\n", classification_report(y_test, y_pred2, zero_division= 0))

Confusion Matrix:
 [[40986   257]
 [    4  2817]]
Accuracy Score: 0.9940767973856209
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.99      1.00     41243
           1       0.92      1.00      0.96      2821

    accuracy                           0.99     44064
   macro avg       0.96      1.00      0.98     44064
weighted avg       0.99      0.99      0.99     44064



In [92]:
gnb = GaussianNB()
gnb.fit(X_train_res, y_train_res)

GaussianNB()

In [93]:
y_pred3 = gnb.predict(X_test)
print(confusion_matrix(y_test,y_pred3))
print(accuracy_score(y_test,y_pred3))
print(classification_report(y_test,y_pred3))

[[40753   490]
 [   34  2787]]
0.9881082062454611
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     41243
           1       0.85      0.99      0.91      2821

    accuracy                           0.99     44064
   macro avg       0.92      0.99      0.95     44064
weighted avg       0.99      0.99      0.99     44064



In [94]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train_res, y_train_res)

KNeighborsClassifier(n_neighbors=3)

In [95]:
y_pred4 = knn.predict(X_test)
print(confusion_matrix(y_test,y_pred4))
print(accuracy_score(y_test,y_pred4))
print(classification_report(y_test,y_pred4))

[[41222    21]
 [    0  2821]]
0.9995234204793029
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     41243
           1       0.99      1.00      1.00      2821

    accuracy                           1.00     44064
   macro avg       1.00      1.00      1.00     44064
weighted avg       1.00      1.00      1.00     44064



In [99]:
xgb = XGBClassifier()
xgb.fit(X_train_res, y_train_res)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [100]:
y_pred6 = xgb.predict(X_test)
print(confusion_matrix(y_test,y_pred6))
print(accuracy_score(y_test,y_pred6))
print(classification_report(y_test,y_pred6))

[[41241     2]
 [    1  2820]]
0.999931917211329
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     41243
           1       1.00      1.00      1.00      2821

    accuracy                           1.00     44064
   macro avg       1.00      1.00      1.00     44064
weighted avg       1.00      1.00      1.00     44064



In [101]:
svc = SVC()
svc.fit(X_train_res, y_train_res)

SVC()

In [102]:
y_pred5 = svc.predict(X_test)
print(confusion_matrix(y_test,y_pred5))
print(accuracy_score(y_test,y_pred5))
print(classification_report(y_test,y_pred5))

[[40912   331]
 [    3  2818]]
0.992420116194626
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     41243
           1       0.89      1.00      0.94      2821

    accuracy                           0.99     44064
   macro avg       0.95      1.00      0.97     44064
weighted avg       0.99      0.99      0.99     44064



In [111]:
#Savign the model as .pkl for XGBOOST 
dump(xgb, "xgb.pkl")


['xgb.pkl']

In [112]:
forecast_df.columns

Index(['timestamp', 'sensor_00', 'sensor_01', 'sensor_02', 'sensor_03',
       'sensor_04', 'sensor_05', 'sensor_06', 'sensor_07', 'sensor_08',
       'sensor_09', 'sensor_10', 'sensor_11', 'sensor_12', 'sensor_13',
       'sensor_14', 'sensor_16', 'sensor_17', 'sensor_18', 'sensor_19',
       'sensor_20', 'sensor_21', 'sensor_22', 'sensor_23', 'sensor_24',
       'sensor_25', 'sensor_26', 'sensor_27', 'sensor_28', 'sensor_29',
       'sensor_30', 'sensor_31', 'sensor_32', 'sensor_33', 'sensor_34',
       'sensor_35', 'sensor_36', 'sensor_37', 'sensor_38', 'sensor_39',
       'sensor_40', 'sensor_41', 'sensor_42', 'sensor_43', 'sensor_44',
       'sensor_45', 'sensor_46', 'sensor_47', 'sensor_48', 'sensor_49',
       'sensor_51', 'machine_status'],
      dtype='object')

In [115]:
X_forecasted = forecast_df.drop(['timestamp','machine_status'], axis=1)

In [117]:
y_pred_xgb = xgb.predict(X_forecasted)

print("Predicted values for XGBoost model:")
print(y_pred_xgb)

Predicted values for XGBoost model:
[0 0 0 ... 0 0 0]


In [118]:
y_pred_xgb.shape

(1440,)

In [121]:
forecast_df['label'] = y_pred_xgb
forecast_df = forecast_df.rename(columns={'label': 'machine_status'})

In [127]:
Local_date = datetime.now().strftime('%Y-%m-%d')
file_path = f"Forecast_Predict_{Local_date}.csv"

forecast_df.to_csv(file_path, index=False)